In [27]:
import pandas as pd
import numpy as np
from pathlib import Path

# ============================================================================
# CONFIGURATION - UPDATE THESE VALUES
# ============================================================================

INPUT_FILE = "load_dump.csv"           # Input CSV/Excel file
TARGET_YEARS = None                    # None (all), 2025 (single), or [2024, 2025] (multiple)
OUTPUT_FILE = None                     # Auto-generated if None

# Zone to column name mapping
ZONE_MAPPING = {
    1: "TPCODL Demand",
    2: "TPWODL Demand",
    3: "TPNODL Demand",
    4: "TPSOSDL Demand",
    5: "Total Demand (as recorded)"
}

print("✅ Configuration loaded")
print(f"   Input: {INPUT_FILE}")
print(f"   Years: {TARGET_YEARS if TARGET_YEARS else 'All'}")


✅ Configuration loaded
   Input: load_dump.csv
   Years: All


In [28]:
# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def load_and_prepare_data(file_path, target_years=None):
    """Load CSV/Excel file and filter by years."""
    print(f"📂 Loading data from {file_path}...")
    
    if file_path.endswith('.xlsx') or file_path.endswith('.xls'):
        df = pd.read_excel(file_path)
    else:
        df = pd.read_csv(file_path)
    
    df["time"] = pd.to_datetime(df["time"], errors="coerce")
    
    available_years = sorted(df['time'].dt.year.unique())
    print(f"✅ Data loaded. Available years: {available_years}")
    
    if target_years is not None:
        if isinstance(target_years, int):
            target_years = [target_years]
        
        df_filtered = df[df["time"].dt.year.isin(target_years)].copy()
        years_str = ", ".join(map(str, target_years))
        print(f"✅ Filtered for year(s): {years_str}")
    else:
        df_filtered = df.copy()
        print(f"✅ Keeping all years")
    
    date_min = df_filtered["time"].min()
    date_max = df_filtered["time"].max()
    print(f"   Date range: {date_min} to {date_max}")
    print(f"   Total records: {len(df_filtered)}")
    
    return df_filtered

def pivot_and_rename(df, zone_mapping):
    """Pivot data by zone and rename columns."""
    print("🔄 Pivoting table...")
    df_pivot = df.pivot_table(index="time", columns="zone_id", values="load")
    
    print("📝 Renaming columns...")
    df_pivot.rename(columns=zone_mapping, inplace=True)
    df_pivot.index.name = "Timestamp"
    df_pivot.reset_index(inplace=True)
    
    print(f"✅ Pivot complete. Shape: {df_pivot.shape}")
    print(f"   Columns: {list(df_pivot.columns)}")
    
    return df_pivot

def fill_zeros(df, numeric_only=True):
    """Replace zeros with NaN and forward-fill."""
    print("🔧 Filling zero values...")
    df = df.copy()
    
    if numeric_only:
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        df[numeric_cols] = df[numeric_cols].replace(0, np.nan).fillna(method='ffill')
    else:
        df = df.replace(0, np.nan).fillna(method='ffill')
    
    print("✅ Zero values replaced and forward-filled")
    return df

def round_numeric_columns(df, decimals=2):
    """Round all numeric columns."""
    print(f"🔢 Rounding to {decimals} decimals...")
    df = df.copy()
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].round(decimals)
    print(f"✅ Rounded {len(numeric_cols)} columns")
    
    return df

def save_data(df, output_path=None, input_file="load.xlsx"):
    """Save processed data to CSV."""
    if output_path is None:
        stem = Path(input_file).stem
        output_path = f"{stem}_processed.csv"
    
    df.to_csv(output_path, index=False)
    print(f"💾 Data saved to: {output_path}")
    
    return output_path

print("✅ Helper functions loaded")


✅ Helper functions loaded


In [29]:
# ============================================================================
# MAIN PIPELINE - Load, Transform, Save
# ============================================================================

print("\n" + "="*70)
print("PROCESSING PIPELINE")
print("="*70)

# Step 1: Load and prepare
df = load_and_prepare_data(INPUT_FILE, TARGET_YEARS)

# Step 2: Pivot and rename
df = pivot_and_rename(df, ZONE_MAPPING)

# Step 3: OPTIONAL - Uncomment to fill zeros
# df = fill_zeros(df)

# Step 4: OPTIONAL - Uncomment to round values
# df = round_numeric_columns(df, decimals=2)

# Step 5: Save
output_file = save_data(df, OUTPUT_FILE, INPUT_FILE)

# Display preview
print("\n📊 Data Preview:")
print(df.head())
print(f"\n✨ Processing complete!")



PROCESSING PIPELINE
📂 Loading data from load_dump.csv...
✅ Data loaded. Available years: [np.int32(2024), np.int32(2025), np.int32(2026)]
✅ Keeping all years
   Date range: 2024-06-04 15:15:00 to 2026-01-15 09:00:00
   Total records: 237829
🔄 Pivoting table...
📝 Renaming columns...
✅ Pivot complete. Shape: (47441, 6)
   Columns: ['Timestamp', 'TPCODL Demand', 'TPWODL Demand', 'TPNODL Demand', 'TPSOSDL Demand', 'Total Demand (as recorded)']
💾 Data saved to: load_dump_processed.csv

📊 Data Preview:
zone_id           Timestamp  TPCODL Demand  TPWODL Demand  TPNODL Demand  \
0       2024-06-04 15:15:00         1822.0         1582.0         1042.0   
1       2024-06-04 15:30:00         1845.0         1556.0         1064.0   
2       2024-06-04 15:45:00         1830.0         1562.0         1077.0   
3       2024-06-04 16:00:00         1865.0         1557.0         1033.0   
4       2024-06-04 16:15:00         1860.0         1566.0         1047.0   

zone_id  TPSOSDL Demand  Total Demand (a

In [31]:
# ============================================================================
# FEATURE: Concatenate Files (Parent + New Data)
# ============================================================================
# Use this cell to merge a parent file (existing data) with new data

def concatenate_files(parent_file, new_file, output_path=None, 
                     parent_date_format=None, new_date_format=None,
                     remove_duplicates=True):
    """
    Merge parent file with new data file.
    Appends only new rows after parent file's last timestamp.
    
    Parameters:
    -----------
    parent_file : str
        Path to parent CSV file
    new_file : str
        Path to new CSV file
    output_path : str
        Output file path (optional)
    parent_date_format : str
        Date format for parent file (e.g., '%d-%m-%Y %H:%M')
        If None, will try auto-detection
    new_date_format : str
        Date format for new file (e.g., '%Y-%m-%d %H:%M:%S')
        If None, will try auto-detection
    remove_duplicates : bool
        Remove duplicate timestamps
    """
    print("\n" + "="*70)
    print("CONCATENATING FILES")
    print("="*70)
    
    print(f"\n📂 Loading parent file: {parent_file}...")
    df_parent = pd.read_csv(parent_file)
    print(f"   Sample timestamps from parent file:")
    print(f"   {df_parent['Timestamp'].head(3).values}")
    
    print(f"\n📂 Loading new file: {new_file}...")
    df_new = pd.read_csv(new_file)
    print(f"   Sample timestamps from new file:")
    print(f"   {df_new['Timestamp'].head(3).values}")
    
    # Parse parent file timestamps
    if parent_date_format:
        print(f"\n⏰ Parsing PARENT file with format: {parent_date_format}")
        df_parent["Timestamp"] = pd.to_datetime(df_parent["Timestamp"], 
                                                format=parent_date_format, 
                                                errors="coerce")
    else:
        print(f"\n⏰ Auto-detecting PARENT file format...")
        df_parent["Timestamp"] = pd.to_datetime(df_parent["Timestamp"], 
                                                errors="coerce")
    
    # Parse new file timestamps
    if new_date_format:
        print(f"⏰ Parsing NEW file with format: {new_date_format}")
        df_new["Timestamp"] = pd.to_datetime(df_new["Timestamp"], 
                                            format=new_date_format, 
                                            errors="coerce")
    else:
        print(f"⏰ Auto-detecting NEW file format...")
        df_new["Timestamp"] = pd.to_datetime(df_new["Timestamp"], 
                                            errors="coerce")
    
    # Check for parsing errors
    parent_nans = df_parent["Timestamp"].isna().sum()
    new_nans = df_new["Timestamp"].isna().sum()
    
    if parent_nans > 0 or new_nans > 0:
        print(f"\n⚠️  WARNING: Failed to parse some timestamps!")
        if parent_nans > 0:
            print(f"   Parent: {parent_nans} unparseable timestamps")
            print(f"   Failed samples: {df_parent[df_parent['Timestamp'].isna()]['Timestamp'].head(3).values}")
        if new_nans > 0:
            print(f"   New: {new_nans} unparseable timestamps")
            print(f"   Failed samples: {df_new[df_new['Timestamp'].isna()]['Timestamp'].head(3).values}")
    
    # Show parsed timestamps
    print(f"\n✅ Timestamps parsed successfully")
    print(f"   Parent: {df_parent['Timestamp'].min()} to {df_parent['Timestamp'].max()}")
    print(f"   New: {df_new['Timestamp'].min()} to {df_new['Timestamp'].max()}")
    
    print(f"\n📊 File sizes:")
    print(f"   Parent: {len(df_parent)} rows")
    print(f"   New: {len(df_new)} rows")
    
    # Get cutoff timestamp from parent file
    parent_last_ts = df_parent["Timestamp"].max()
    print(f"\n📅 Parent file ends at: {parent_last_ts}")
    
    # Filter new data - only keep rows AFTER parent's last timestamp
    df_new_filtered = df_new[df_new["Timestamp"] > parent_last_ts].copy()
    rows_to_append = len(df_new_filtered)
    
    if rows_to_append == 0:
        print(f"\n⚠️  WARNING: No new data found after {parent_last_ts}")
        print(f"   New file date range: {df_new['Timestamp'].min()} to {df_new['Timestamp'].max()}")
        print(f"   Parent ends in {df_parent['Timestamp'].max().year}, but new file starts in {df_new['Timestamp'].min().year}")
        print(f"   Make sure the files have overlapping or sequential date ranges!")
        return None
    else:
        print(f"   Will append: {rows_to_append} new rows")
    
    # Combine dataframes
    df_combined = pd.concat([df_parent, df_new_filtered], ignore_index=True)
    
    # Remove duplicates if requested
    if remove_duplicates:
        initial_len = len(df_combined)
        df_combined = df_combined.drop_duplicates(subset=["Timestamp"], keep="first")
        removed = initial_len - len(df_combined)
        if removed > 0:
            print(f"🗑️  Removed {removed} duplicate timestamps")
    
    print(f"\n✅ Concatenation complete!")
    print(f"   Final size: {len(df_combined)} rows")
    print(f"   Date range: {df_combined['Timestamp'].min()} to {df_combined['Timestamp'].max()}")
    
    # Save if output path provided
    if output_path:
        df_combined.to_csv(output_path, index=False)
        print(f"💾 Saved to: {output_path}")
    
    return df_combined

# EXAMPLE USAGE:
# Parent file format: DD-MM-YYYY HH:MM (01-01-2017 00:00)
# New file format: YYYY-MM-DD HH:MM:SS (2024-06-04 15:15:00)

parent_file = "2025_till_31_march.csv"
new_file = "load_dump_processed_filled.csv"
df_combined = concatenate_files(
    parent_file, 
    new_file,
    output_path="2026_proper_data.csv",
    parent_date_format="%d-%m-%Y %H:%M",      # Parent: DD-MM-YYYY HH:MM
    new_date_format="%Y-%m-%d %H:%M:%S"       # New: YYYY-MM-DD HH:MM:SS
)

# COMMON DATE FORMATS:
# "%d-%m-%Y %H:%M"         -> 01-01-2017 00:00
# "%Y-%m-%d %H:%M:%S"      -> 2024-06-04 15:15:00
# "%m-%d-%Y %H:%M:%S"      -> 06-04-2024 15:15:00
# "%d/%m/%Y %H:%M"         -> 01/01/2017 00:00



CONCATENATING FILES

📂 Loading parent file: 2025_till_31_march.csv...
   Sample timestamps from parent file:
   ['01-01-2017 00:00' '01-01-2017 00:15' '01-01-2017 00:30']

📂 Loading new file: load_dump_processed_filled.csv...
   Sample timestamps from new file:
   ['2024-06-04 15:15:00' '2024-06-04 15:30:00' '2024-06-04 15:45:00']

⏰ Parsing PARENT file with format: %d-%m-%Y %H:%M
⏰ Parsing NEW file with format: %Y-%m-%d %H:%M:%S

✅ Timestamps parsed successfully
   Parent: 2017-01-01 00:00:00 to 2025-03-31 10:45:00
   New: 2024-06-04 15:15:00 to 2026-01-15 09:00:00

📊 File sizes:
   Parent: 289100 rows
   New: 56616 rows

📅 Parent file ends at: 2025-03-31 10:45:00
   Will append: 27833 new rows

✅ Concatenation complete!
   Final size: 316933 rows
   Date range: 2017-01-01 00:00:00 to 2026-01-15 09:00:00
💾 Saved to: 2026_proper_data.csv


In [26]:
# ============================================================================
# ADVANCED: Load & Transform Existing Data
# ============================================================================
# Use this cell to work with already-processed CSV files

# Example 1: Load processed CSV and apply additional transformations
# csv_file = "2025_processed_data.csv"
# df_loaded = pd.read_csv(csv_file)
# df_loaded["Timestamp"] = pd.to_datetime(df_loaded["Timestamp"])

# # Apply transformations
# df_transformed = fill_zeros(df_loaded)
# df_transformed = round_numeric_columns(df_transformed, decimals=2)

# # Save
# save_data(df_transformed, "2025_processed_cleaned.csv")

# Example 2: Load and check data
# df_check = pd.read_csv("my_file.csv")
# print(f"Shape: {df_check.shape}")
# print(f"Columns: {list(df_check.columns)}")
# print(df_check.head())


# Data Load & Converter

Complete pipeline for loading, converting, and processing forecasting data with concatenation support.

## Quick Start

1. **Cell 2** - Set your configuration (file names, years, etc.)
2. **Cell 3** - Load and process data
3. **Cell 4** - (Optional) Concatenate with parent file
4. **Cell 5** - (Optional) Advanced transformations

## Features

| Feature | Description |
|---------|------------|
| **Load Data** | Parse CSV/Excel with datetime support |
| **Filter Years** | Keep all, single, or multiple years |
| **Pivot & Rename** | Reorganize zones into readable columns |
| **Fill Zeros** | Replace zeros with forward-filled values |
| **Round Values** | Round numeric columns to N decimals |
| **Concatenate Files** | Merge parent file + new data seamlessly |
| **Auto Output** | Generated filenames based on input |
